# Import Module

In [1]:
import sqlite3
import pandas as pd
import time

In [2]:
def connect_database(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as error:
        print(error)
    return conn

# Task 1

In [3]:
def task1():
    db1 = 'dheafa_task1_python.db'
    create_table1 = '''create table transactions(
    id uuid,
    tx_amount float,
    tx_type varchar(10),
    status varchar(10),
    insert_time DATETIME DEFAULT CURRENT_TIMESTAMP
    )'''
    insert_value1 = '''insert into transactions(id, tx_amount, tx_type, status) values
    (102, 20, 'buy', 'open'),
    (111, 30, 'buy', 'open'),
    (111, 15, 'sell', 'close'),
    (121, 50, 'buy', 'close'),
    (160, 10, 'sell', 'open')
    '''
    recent_time = '''select id, tx_amount, tx_type, status, max(insert_time) as recent_time
    from transactions
    group by 1
    order by 1,5
    '''
    conn = connect_database(db1)
    c = conn.cursor()
    c.execute("drop table if exists transactions")
    if conn is not None:
        c.execute(create_table1)
        for i in range (1,5):
            c.execute(insert_value1)
            time.sleep(3)
        conn.commit()
        df1 = pd.read_sql(recent_time, conn)
        print(df1)
    else:
        print("Error!")
task1()

    id  tx_amount tx_type status          recent_time
0  102       20.0     buy   open  2021-06-26 10:03:13
1  111       30.0     buy   open  2021-06-26 10:03:13
2  121       50.0     buy  close  2021-06-26 10:03:13
3  160       10.0    sell   open  2021-06-26 10:03:13


# Task 2

In [4]:
def task2():
    db2 = 'dheafa_task2_python.db'
    create_table2 = '''create table sales(
    date DATE,
    sales float
    )'''
    insert_value2 = '''insert into sales values
    ('01/01/2021', 100), ('02/01/2021', 34), ('02/01/2021', 123), ('02/01/2021', 134),
    ('03/01/2021', 145), ('03/01/2021', 24), ('04/01/2021', 541), ('04/01/2021', 636),
    ('05/01/2021', 322), ('06/01/2021', 242), ('07/01/2021', 22), ('08/01/2021', 46)
    '''
    rolling_sum = '''select date, sales, sum(sales) over (order by date) as rolling_sum
    from sales
    '''
    conn = connect_database(db2)
    c = conn.cursor()
    c.execute("drop table if exists sales")
    if conn is not None:
        c.execute(create_table2)
        c.execute(insert_value2)
        c.execute(rolling_sum)
        df2 = pd.read_sql(rolling_sum, conn)
        print(df2)
    else:
        print("Error!")
task2()

          date  sales  rolling_sum
0   01/01/2021  100.0        100.0
1   02/01/2021   34.0        391.0
2   02/01/2021  123.0        391.0
3   02/01/2021  134.0        391.0
4   03/01/2021  145.0        560.0
5   03/01/2021   24.0        560.0
6   04/01/2021  541.0       1737.0
7   04/01/2021  636.0       1737.0
8   05/01/2021  322.0       2059.0
9   06/01/2021  242.0       2301.0
10  07/01/2021   22.0       2323.0
11  08/01/2021   46.0       2369.0


# Task 3

In [5]:
data = [
    {
        "user" : {
            "id" : 1,
            "name" : "agus",
            "email" : "agus@example.com"
        },
        "txs" : [
            {
                "id" : 1, 
                "tx_date" : "2020-01-01",
                "tx_amount" : 10,
                "tx_type" : "buy"
            },
            {
                "id" : 3,
                "tx_date" : "2020-02-01",
                "tx_amount" : 10,
                "tx_type" : "buy"
            },
            {
                "id" : 5,
                "tx_date" : "2020-03-01",
                "tx_amount" : 20,
                "tx_type" : "sell"
            }
        ]
    },
    {
        "user" : {
            "id" : 2,
            "name" : "asep",
            "email" : "asep@example.com"
        },
        "txs" : [
            {
                "id" : 2,
                "tx_date" : "2020-01-01",
                "tx_amount" : 10,
                "tx_type" : "sell"
            },
            {
                "id" : 4,
                "tx_date" : "2020-02-01",
                "tx_amount" : 10,
                "tx_type" : "sell" 
            },
            {
                "id" : 6,
                "tx_date" : "2020-03-01",
                "tx_amount" : 20,
                "tx_type" : "buy"  
            }
        ]
    }
]

df = pd.json_normalize(data, 'txs', [['user','id'],['user','name'],['user','email']])
CSV_1 = df[['user.id','id']]
CSV_1.to_csv('CSV_1.csv', header=['user_id','tsx_id'], index=False)
CSV_2 = df[['user.id','user.name','user.email']].drop_duplicates()
CSV_2.to_csv('CSV_2.csv', header=['id','name','email'], index=False)
CSV_3 = df[['id','tx_date','tx_amount','tx_type']].sort_values('id')
CSV_3.to_csv('CSV_3.csv', header=['id','tx_date','tx_amount','tx_type'], index=False)